# Tutorial data

In [ ]:
# import pandas as pd

# data_path = "data/train/day_0_0.001_train.csv"

# df = pd.read_csv(data_path, header=None)
# df.head()

In [ ]:
# example = df.loc[1, 0]
# example

In [ ]:
# features = example.split('\t')
# len(features)
# features

# Read s3 orc data

In [2]:
column_names = []
with open(f'./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        column_names.append(line.split(' ')[1].strip())
print(column_names)

['_11001', '_11002', '_11003', '_11004', '_11007', '_11008', '_11021', '_11022', '_11023', '_11024', '_11041', '_11042', '_11043', '_11044', '_11045', '_11046', '_11061', '_11062', '_11063', '_11064', '_11065', '_11066', '_11081', '_11082', '_11083', '_11084', '_11085', '_11086', '_11601', '_11602', '_11603', '_12001', '_12002', '_12003', '_12004', '_12005', '_12006', '_20001', '_20002', '_20003', '_20101', '_20102', '_20201', '_20202', '_20203', '_20204', '_20205', '_20206', '_20207', '_20208', '_20209', '_20210', '_30001', '_30002', '_30003', '_30004', '_30005', '_30006', '_30201', '_30202', '_30203', '_30204', '_30205', '_30206', '_30207', '_40001', '_40002', '_40003', '_40004', '_40005', '_40201', '_40202', '_40203', '_40204', '_40205', '_40206', '_40207', '_40208', '_40209', '_40210', '_40211', '_40212', '_40213', '_40214', '_40215', '_40231', '_40301', '_40302', '_40303', '_40304', '_40305', '_40306', '_40307', '_40321', '_40322', '_40323', '_40324', '_50801', '_50802', '_50805',

In [3]:
import metaspore as ms

# train_dataset_path = ROOT_DIR + '/data/train/day_0_0.001_train.csv'

file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
num_files = 10
file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
train_dataset_path = [file_base_path + file_name for file_name in file_names]

# # train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/part-00000-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc'
# train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
# train_dataset_path = [
#     's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/',
#     's3://mv-mtg-di-for-poc-datalab/2024/06/14/01/',
#     's3://mv-mtg-di-for-poc-datalab/2024/06/14/02/',
# ]

# train_dataset_path = ['/home/ubuntu/data/processed_parquets/part-00000-03f00bdf-801e-4399-8604-d26e898a24f5-c000.snappy.parquet']

spark_confs = {
    'spark.eventLog.enabled':'true',
    'spark.executor.memory': '10g',
    'spark.driver.memory': '10g',
    "spark.driver.maxResultSize": "10g",
    "spark.sql.files.ignoreCorruptFiles": "true"
}

spark_session = ms.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=1,
                                     server_count=0,
                                     log_level='WARN',
                                     spark_confs=spark_confs)


train_dataset = ms.input.read_s3_csv(spark_session, 
                                    train_dataset_path, 
                                    format='orc',
                                    shuffle=False,
                                    delimiter='\t',
                                    multivalue_delimiter="\001", 
                                    column_names=column_names,
                                    multivalue_column_names=column_names[:-1])

# train_dataset.printSchema()
# train_dataset.count()

In [ ]:
pd_df = train_dataset.toPandas()

In [17]:
import time
import pandas as pd
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType, FloatType
import numpy as np

# def process_udf(iterator):
#     for minibatch in iterator:
#         minibatch = 
#         yield minibatch


def split_value_weight(minibatch):
    start_time = time.time()
    def split(items):
        values = []
        weights = []
        for item in items:
            if '\003' in item:
                value, weight = item.split('\003')
            else:
                value, weight = '', '0'
            values.append(value)
            weights.append(float(weight))
        return values, weights

    values_dict = {}
    weights_dict = {}

    for column in minibatch.columns:
        if column == 'label':
            continue
        values_dict[f'{column}'] = []
        weights_dict[f'{column}_weight'] = []
        for items in minibatch[column]:
            try:
                # items is expected to be a numpy array or list
                if isinstance(items, (np.ndarray, list)):
                    values, weights = split(items)
                else:
                    values, weights = split([items])
                values_dict[f'{column}'].append(values)
                weights_dict[f'{column}_weight'].append(weights)
            except Exception as e:
                print(f"split_value_weight error, {e}, items: {items}")
                values_dict[f'{column}'].append([])
                weights_dict[f'{column}_weight'].append([])

    minibatch_value = pd.DataFrame(values_dict)
    minibatch_weight = pd.DataFrame(weights_dict)

    end_time = time.time()
    split_duration = end_time - start_time
    # print(f"Type of minibatch: {type(minibatch)}, {minibatch.shape}, split duration: {split_duration:.3f} s.")

    return pd.concat([minibatch_value, minibatch_weight, minibatch[['label']]], axis=1)


def process_minibatch(iterator):
    for minibatch in iterator:
        yield minibatch
        # yield split_value_weight(minibatch)

def generate_output_schema(columns):
    schema_fields = []
    for column in columns:
        if column == 'label':
            continue
        schema_fields.append(StructField(f'{column}', ArrayType(StringType()), True))
        schema_fields.append(StructField(f'{column}_weight', ArrayType(FloatType()), True))

    schema_fields.append(StructField('label', StringType(), True))
    return StructType(schema_fields)

start_time = time.time()
# output_schema = generate_output_schema(train_dataset.columns)
output_schema = train_dataset.schema
df = train_dataset.mapInPandas(process_minibatch, output_schema)
df.collect()
duration = time.time() - start_time
print(f"Duration: {duration:.3f} s.")

Duration: 160.865 s.


In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("ReadParquetData") \
    .getOrCreate()

# Path to the Parquet files
parquet_path = '/home/ubuntu/data/processed_parquets/part-00000-03f00bdf-801e-4399-8604-d26e898a24f5-c000.snappy.parquet'

# Read the Parquet file into a DataFrame
df = spark.read.format("parquet").load(parquet_path)

# Show the DataFrame schema and content
df.printSchema()
df.show()

In [ ]:
pd_df = df.toPandas()

In [ ]:
pd_df.head(5)

In [ ]:
columns_weight = [f'{column}_weight' for column in column_names if column!='label']
df_weights = pd_df[columns_weight]
df_weights.head(5)

In [ ]:
np_weights = df_weights[:1000].to_numpy()
flat_weights = np.hstack(np_weights.ravel())
flat_weights.shape

In [ ]:
a = pd_df[['_11001']]
print(type(a))
a.head()


In [ ]:
row_index = 1
column_name = '_11001_weight'

# Use take() to retrieve the specific row
row = df.select(column_name).take(row_index + 1)[-1]  # take(row_index + 1) gets row 10
value = row[column_name]

print(f"Value at column '{column_name}', row {row_index}: {value}")

In [ ]:
train_dataset.show(5)

In [ ]:
from tqdm import tqdm
import pandas as pd
pd.set_option("display.max_columns", None)

columns = column_names[:-1]
feature_hash = {}
feature_value = {}

num_files_per_chunk = 10
num_chunks = (num_files + num_files_per_chunk - 1) // num_files_per_chunk

num_rows = 0

for i in tqdm(range(num_chunks), total=num_chunks):
    chunk_path = train_dataset_path[i*num_files_per_chunk:(i+1)*num_files_per_chunk]

    # Read the chunk
    train_dataset = ms.input.read_s3_csv(spark_session, 
                                     chunk_path, 
                                     format='orc',
                                     shuffle=False,
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=column_names,
                                     multivalue_column_names=column_names[:-1])
    
    # Convert the chunk to a Pandas DataFrame
    df = train_dataset.toPandas()
    # df.head()
    num_rows += df.shape[0]

    # num_positive = df[df['label']=='1'].shape[0]
    # num_negative = df[df['label']=='0'].shape[0]
    # print(f"num_positive: {num_positive}, num_negative: {num_negative}, ratio: {num_positive/num_negative*100}")

    for i, row in df.iterrows(): # tqdm(df.iterrows(), total=df.shape[0]):
        for column in columns[:-1]:
            if column in feature_value:
                feature_value[column].append(row[column].shape[0])
            else:
                feature_value[column] = [row[column].shape[0]]
            
            for item in row[column]:
                if '\003' in item:
                    hash, weight = item.split('\003')
                else:
                    hash, weight = '', 0
                if column in feature_hash:
                    feature_hash[column].add(hash)
                else:
                    feature_hash[column] = set(hash)

print(f"Number of files: {num_files}, number of rows: {num_rows}")

In [ ]:
df_feature = pd.DataFrame(feature_value)
df_feature.describe()

In [ ]:
feature_hash_count = dict([(k, [len(v)]) for k, v in feature_hash.items()])
df_hash = pd.DataFrame(feature_hash_count)
df_hash

In [ ]:
hashes = set()

for k, v in feature_hash.items():
    hashes |= v
print(f"Number of total hashes: {len(hashes)}")

In [ ]:
total_hash = df_hash.sum(axis=1).values[0]
print(f"Number of total hashes: {total_hash}")

# Use pyspark to analyze

In [ ]:
columns = []
with open(f'./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        columns.append(line.split(' ')[1].strip())
print(columns)

In [ ]:
import metaspore as ms
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, explode, sum as spark_sum, countDistinct
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType

# Read column names from file
columns = []
with open('./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        columns.append(line.split(' ')[1].strip())
print(columns)

# Define Spark configuration and session
file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
num_files = 1
file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
train_dataset_path = [file_base_path + file_name for file_name in file_names]

spark_confs = {
    'spark.eventLog.enabled': 'true',
    'spark.executor.memory': '10g',
    'spark.driver.memory': '10g',
    "spark.driver.maxResultSize": "10g",
    "spark.sql.files.ignoreCorruptFiles": "true"
}

spark_session = ms.spark.get_session(local=True,
                                     batch_size=100,
                                     worker_count=3,
                                     server_count=3,
                                     log_level='WARN',
                                     spark_confs=spark_confs)

# Load the dataset
train_dataset = ms.input.read_s3_csv(spark_session, 
                                     train_dataset_path, 
                                     format='orc',
                                     shuffle=False,
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=columns,
                                     multivalue_column_names=columns[:-1])

print(f"Number of orcs: {num_files}, total number of rows: {train_dataset.count()}")

# Define UDF to process rows
def process_row(column_values):
    feature_hashes = set()
    for value in column_values:
        if '\003' in value:
            hash_val, weight = value.split('\003')
        else:
            hash_val, weight = '', 0
        feature_hashes.add(hash_val)

    feature_values = len(column_values)
    return (feature_values, list(feature_hashes))

# Define schema for UDF output
schema = StructType([
    StructField("feature_values", IntegerType(), True),
    StructField("hashes", ArrayType(StringType()), True)
])

# Register the UDF
process_row_udf = udf(process_row, schema)

# Apply UDF on each column and process the dataset
for column in columns[:-1]:
    train_dataset = train_dataset.withColumn(f'{column}_processed', process_row_udf(col(column)))

train_dataset.show()


In [ ]:
# Extract the processed columns
result_columns = [f'{column}_processed' for column in columns[:-1]]
processed_df = train_dataset.select(*result_columns)
processed_df.show()

In [ ]:
# Select the desired row and column
row_value = train_dataset.select('_11002_processed').collect()[4][0]

# Print the value
print(row_value)

In [ ]:
# 1. Obtain the percentiles of feature_values using approxQuantile
from tqdm import tqdm
percentiles = [0.25, 0.5, 0.75, 1.0]

for col_name in tqdm(processed_df.columns, total=len(processed_df.columns)):
    feature_values_df = processed_df.select(col(f"{col_name}.feature_values").cast("double").alias("feature_values"))
    
    # Print schema to verify cast
    # feature_values_df.printSchema()
    
    # Show data to verify cast
    # feature_values_df.show()
    
    # Verify the data type of the column
    # data_type = feature_values_df.schema["feature_values"].dataType
    # print(f"Data type of feature_values in {col_name}: {data_type}")
    
    # # Check for null values or any anomalies in the column
    # null_count = feature_values_df.filter(col("feature_values").isNull()).count()
    # print(f"Number of null values in feature_values of {col_name}: {null_count}")
    
    # Obtain approximate percentiles
    try:
        percentile_values = feature_values_df.approxQuantile("feature_values", percentiles, 0.01)
        # print(f"Percentiles for {col_name}: {percentile_values}")
    except Exception as e:
        print(f"Error occurred while calculating percentiles for {col_name}: {e}")

# 2. Obtain the total number of different hash values for each column and across all columns
distinct_hash_counts = {}
all_hashes_df = []

print(f"Starting counting hashes")
for col_name in tqdm(processed_df.columns, total=len(processed_df.columns)):
    hashes_df = processed_df.select(explode(col(f"{col_name}.hashes")).alias("hashes"))
    distinct_count = hashes_df.select(countDistinct("hashes")).first()[0]
    distinct_hash_counts[col_name] = distinct_count
    all_hashes_df.append(hashes_df)

union_hashes_df = all_hashes_df[0]
for df in all_hashes_df[1:]:
    union_hashes_df = union_hashes_df.union(df)

total_distinct_hashes = union_hashes_df.select(countDistinct("hashes")).first()[0]

print(f"Distinct hash counts per column: {distinct_hash_counts}")
print(f"Total distinct hashes across all columns: {total_distinct_hashes}")

# Stop the Spark session
# spark.stop()

# To do data approximation

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Given data
number_of_files = np.array([1, 10, 258416]).reshape(-1, 1)
number_of_hashes = np.array([1959220, 9758165, 10937500000])

# Logarithm fit for the given data
x_log = np.log10(number_of_files)
y_log = np.log10(number_of_hashes)

# Fit the linear model on the log-transformed data
log_model = LinearRegression()
log_model.fit(x_log, y_log)

# Predict missing values on the log scale
missing_files = np.array([2148, 35787]).reshape(-1, 1)
missing_files_log = np.log10(missing_files)
predicted_hashes_log = log_model.predict(missing_files_log)
predicted_hashes = 10 ** predicted_hashes_log  # convert back to the original scale

# Print predicted values
predicted_hashes_dict = {n_files: int(n_hashes) for n_files, n_hashes in zip(missing_files.flatten(), predicted_hashes)}
print(predicted_hashes_dict)

# Plotting the fit and the predictions
plt.figure(figsize=(10, 6))
plt.scatter(number_of_files, number_of_hashes, color='blue', label='Given Data')
plt.scatter(missing_files, predicted_hashes, color='red', label='Predicted Data')
plt.plot(np.concatenate([number_of_files, missing_files]), 
         10 ** log_model.predict(np.log10(np.concatenate([number_of_files, missing_files]))), 
         color='green', label='Logarithmic Fit')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Number of files')
plt.ylabel('Number of hashes')
plt.legend()
plt.title('Logarithmic Fit for Number of Files vs Number of Hashes')
plt.show()


In [ ]:
2788499930*16*4/10**9